<a href="https://colab.research.google.com/github/manmeet3591/python_class/blob/master/gee/gedi_extract_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import ee, folium

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
!sudo add-apt-repository ppa:ubuntugis/ppa
!sudo apt-get update
!sudo apt-get install gdal-bin libgdal-dev
!pip install -U pip
!pip install rasterio

In [ ]:
try:
    import geopandas as gpd
    import shapefile
    from osgeo import ogr,osr,gdal
except:
    !pip install geopandas
    !pip install PyShp
    !pip install gdal
import geopandas as gpd
import shapefile
import numpy as np
import os
import time
from osgeo import ogr,osr,gdal
import tensorflow as tf

In [ ]:
def qualityMask(im):
    return im.updateMask(im.select('quality_flag').eq(1)).updateMask(im.select('degrade_flag').eq(0))
dataset = ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY').map(qualityMask).select('rh98')

In [ ]:
visualization = {
  'min': 1.0,
  'max': 60.0,
  'palette': ['darkred','red','orange','green','darkgreen']
  #'bands': ['B4', 'B3', 'B2'],
}

map = folium.Map(location=[1.64, 110.20],zoom_start=4, height=1000,width=1500)
mapid = dataset.get(100).getMapId(visualization)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='GEDI',
    opacity=0.5,
  ).add_to(map)

map.add_child(folium.LayerControl())
title_html = '''
             <h3 align="center" style="font-size:10px"><b>'GEDI RH98'</b></h3>
             '''
map.get_root().html.add_child(folium.Element(title_html))
map

In [ ]:
aoi = ee.Geometry.Polygon(
        [[[10.752981103928594, 0.7627966289396413],
          [10.752981103928594, -0.7615114430026542],
          [13.104055322678594, -0.7615114430026542],
          [13.104055322678594, 0.7627966289396413]]], np.nan, False)

In [ ]:
#// GEDI pre-processing
def func_(image):
    return image.updateMask(image.select('sensitivity').gt(0.98)).updateMask(image.select('quality_flag').eq(1)).updateMask(image.select('degrade_flag').eq(0)).select('rh100').toInt()

gedi = ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY').filterBounds(aoi).map(func_)

In [ ]:
projection = gedi.first().projection().aside(print)
scale = projection.nominalScale().aside(print)
mosaic = gedi.mosaic().setDefaultProjection(crs=projection, scale=scale)
mosaicMask = mosaic.lt(60)
mosaic = mosaic.updateMask(mosaicMask)
#//print(mosaic)   


referenceData = mosaic.reduceToVectors(geometry = aoi,
      geometryType= 'polygon',
      scale = scale,
      maxPixels = 100e9,
      tileScale=16,
      crs = projection,
          )

In [ ]:
task = ee.batch.Export.table.toDrive(
            collection= referenceData,
            folder= 'gedi',
            description='referenceData',
            fileFormat= 'SHP'
)
task.start()